In [1]:
# change to tf 2.0 (run on colab)
# !pip uninstall tensorflow
# !pip install tensorflow==2.0.0

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [ ]:
# Load git code
# !git clone -s https://VincentHenric:@github.com/VincentHenric/Deep_learning_project.git
# %cd Deep_learning_project
# !ls

from google.colab import files

In [ ]:
# To use dynamic batches
# !pip install pybind11
# !./compile.sh

In [ ]:
# Access Drive
from google.colab import drive
%cd /content/
drive.mount('/content/gdrive')

# Move data to working folder
# %cp -a "../gdrive/My Drive/deep_learning_project/." "."
# %ls

# Go to Drive folder
%cd gdrive/My Drive/deep_learning_project
%ls

In [3]:
%load_ext autoreload
%autoreload 1

import numpy as np
from matplotlib import pyplot as plt

import os
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
import golois

%aimport models
%aimport utils
%aimport generators

print(tf.__version__)

2.0.0


In [ ]:
PLANES = 8
MOVES = 361
DYNAMIC_BATCH = True
model_name = ""

In [ ]:
if DYNAMIC_BATCH:
#     N = 100000
#     input_data = np.random.randint(2, size=(N, 19, 19, PLANES))
#     input_data = input_data.astype ('float32')
    
#     policy = np.random.randint(MOVES, size=(N,))
#     policy = keras.utils.to_categorical (policy)
    
#     value = np.random.randint(2, size=(N,))
#     value = value.astype ('float32')
    
#     end = np.random.randint(2, size=(N, 19, 19, 2))
#     end = end.astype ('float32')

#     golois.getBatch (input_data, policy, value, end)
    
    generator = generators.GoloisSequence(N=100000,
                                          batch_size=128,
                                          change_batch=5,
                                          planes=PLANES,
                                          moves=MOVES)
    input_data_val = np.load ('input_data.npy')
    policy_val = np.load ('policy.npy')
    value_val = np.load ('value.npy')
    end_val = np.load ('end.npy')
    
else:
    input_data = np.load ('input_data.npy')
    policy = np.load ('policy.npy')
    value = np.load ('value.npy')
    end = np.load ('end.npy')

In [ ]:
models_files = os.listdir('models')
models_files = [file[:-3] for file in models_files if file.endswith('.h5')]
if model_name in models_files:
    print("The model already exists: loading model")
    model = keras.models.load_model('models/{}.h5'.format(model_name))
else:
    print("New model")
    model = models.get_model_week2(128)

model.summary()

model.compile(optimizer='adam',
              loss={'value': 'mse', 'policy': 'categorical_crossentropy'})

In [ ]:
checkpointer = ModelCheckpoint(filepath='models/{}.h5'.format(model_name),
                                   verbose=1,
                                   save_best_only=True)

print('Start learning')
# history = model.fit(input_data, {'policy': policy, 'value': value},
#           epochs=20, batch_size=128, validation_split=0.1,
#           callbacks=[checkpointer])

history = model.fit_generator(generator,
          epochs=20, validation_data=(input_data_val, (policy_val, value_val)),
          callbacks=[checkpointer], shuffle = False)

In [ ]:
fig, ax = plt.subplots(3, figsize=(10,10))

ax[0].plot(history.history['loss'], label="loss")
ax[0].plot(history.history['val_loss'], label = "val loss")
ax[0].legend()
ax[1].plot(history.history['policy_loss'], label="policy loss")
ax[1].plot(history.history['val_policy_loss'], label = "val policy loss")
ax[1].legend()
ax[2].plot(history.history['value_loss'], label="value loss")
ax[2].plot(history.history['val_value_loss'], label = "val value loss")
ax[2].legend()

plt.xlabel('nb epochs')
# plt.title(key)
# plt.savefig('history.png')
plt.show()

In [ ]:
utils.save_history(history.history, model_name, 'a', path='histories')

In [ ]:
# Download the model
files.download('models/{}.h5'.format(model_name))

In [ ]:
# Download the history
files.download('histories/{}.json'.format(model_name))

In [6]:
model = keras.models.load_model('models/HENRIC.h5')